# Loading a PyTorch Model in C++

In [1]:
import torch
import torchvision

## Step 1: Converting Your PyTorch Model to Torch Script

In [2]:
# An instance of your model.
model = torchvision.models.resnet18()

In [3]:
# An example input you would normally provide to your model's forward() method.
example = torch.rand(1, 3, 224, 224)

In [4]:
# Use torch.jit.trace to generate a torch.jit.ScriptModule via tracing.
traced_script_module = torch.jit.trace(model, example)

In [5]:
output = traced_script_module(torch.ones(1, 3, 224, 224))

In [6]:
print(output[0, :5])

tensor([ 0.0808,  0.5042,  0.7030, -0.9465,  0.4538], grad_fn=<SliceBackward>)


## Converting to Torch Script via Annotation

In [7]:
class MyModule(torch.nn.Module):
    def __init__(self, N, M):
        super(MyModule, self).__init__()
        self.weight = torch.nn.Parameter(torch.rand(N, M))

    def forward(self, input):
        if input.sum() > 0:
          output = self.weight.mv(input)
        else:
          output = self.weight + input
        return output

Because the `forward` method of this module uses control flow that is dependent on the input, it is not suitable for tracing. Instead, we can convert it to a `ScriptModule` by subclassing it from `torch.jit.ScriptModule` and adding a `@torch.jit.script_method` annotation to the model’s `forward` method:

In [8]:
class MyModule(torch.jit.ScriptModule):
    def __init__(self, N, M):
        super(MyModule, self).__init__()
        self.weight = torch.nn.Parameter(torch.rand(N, M))

    @torch.jit.script_method
    def forward(self, input):
        if bool(input.sum() > 0):
          output = self.weight.mv(input)
        else:
          output = self.weight + input
        return output

In [9]:
my_script_module = MyModule(2, 3)

## Step 2: Serializing Your Script Module to a File

Once you have a `ScriptModule` in your hands, either from tracing or annotating a PyTorch model, you are ready to serialize it to a file. Later on, you’ll be able to load the module from this file in C++ and execute it without any dependency on Python. Say we want to serialize the `ResNet18` model shown earlier in the tracing example. To perform this serialization, simply call save on the module and pass it a filename:

In [10]:
traced_script_module.save("model.pt")

This will produce a `model.pt` file in your working directory. We have now officially left the realm of Python and are ready to cross over to the sphere of C++.

In [11]:
!ls -lah

total 45M
drwxrwxr-x 3 ubuntu ubuntu 4.0K May 11 02:27 .
drwxrwxr-x 4 ubuntu ubuntu 4.0K May 11 02:26 ..
-rw-rw-r-- 1 ubuntu ubuntu  260 May 10 18:16 CMakeLists.txt
-rw-rw-r-- 1 ubuntu ubuntu 7.3K May 10 17:48 CPP_Export.ipynb
drwxrwxr-x 2 ubuntu ubuntu 4.0K May 11 02:26 .ipynb_checkpoints
-rw-rw-r-- 1 ubuntu ubuntu  45M May 11 02:27 model.pt
-rw-rw-r-- 1 ubuntu ubuntu  796 May 11 02:14 resnet18-app.cpp


## Step 3: Loading Your Script Module in C++

To load your serialized PyTorch model in C++, your application must depend on the PyTorch C++ API – also known as LibTorch. The LibTorch distribution encompasses a collection of shared libraries, header files and CMake build configuration files. While CMake is not a requirement for depending on LibTorch, it is the recommended approach and will be well supported into the future. For this tutorial, we will be building a minimal C++ application using CMake and LibTorch that simply loads and executes a serialized PyTorch model.

### A Minimal C++ Application

In [14]:
!cat resnet18-app.cpp

#include <torch/script.h> // One-stop header.

#include <iostream>
#include <memory>

int main(int argc, const char* argv[]) {
    if (argc != 2) {
        std::cerr << "usage: example-app <path-to-exported-script-module>\n";
        return -1;
    }

    // Deserialize the ScriptModule from a file using torch::jit::load().
    std::shared_ptr<torch::jit::script::Module> module = torch::jit::load(argv[1]);

    assert(module != nullptr);
    std::cout << "ok\n";
}

### Depending on LibTorch and Building the Application

In [15]:
!cat CMakeLists.txt

cmake_minimum_required(VERSION 3.0 FATAL_ERROR)
project(custom_ops)

find_package(Torch REQUIRED)

add_executable(resnet18-app resnet18-app.cpp)
target_link_libraries(resnet18-app "${TORCH_LIBRARIES}")
set_property(TARGET resnet18-app PROPERTY CXX_STANDARD 11)

The last thing we need to build the example application is the LibTorch distribution. You can always grab the latest stable release from the [download page](https://pytorch.org/get-started/locally/#start-locally) on the PyTorch website. If you download and unzip the latest archive, you should receive a folder with the following directory structure:

In [21]:
!ls -l ../../; ls -l ../../libtorch

total 63364
drwxr-xr-x 6 ubuntu ubuntu     4096 Dec  6 07:14 libtorch
-rw-rw-r-- 1 ubuntu ubuntu 64875864 Dec  7 19:14 libtorch-shared-with-deps-1.0.0.zip
drwxrwxr-x 4 ubuntu ubuntu     4096 May 11 02:26 notebooks
total 20
drwxr-xr-x 2 ubuntu ubuntu 4096 Dec  6 07:14 bin
-rw-r--r-- 1 ubuntu ubuntu    6 Dec  6 07:14 build-version
drwxr-xr-x 8 ubuntu ubuntu 4096 Dec  6 07:14 include
drwxr-xr-x 2 ubuntu ubuntu 4096 Dec  6 07:38 lib
drwxr-xr-x 3 ubuntu ubuntu 4096 Dec  6 07:14 share


- The `lib/` folder contains the shared libraries you must link against,
- The `include/` folder contains header files your program will need to include,
- The `share/` folder contains the necessary CMake configuration to enable the simple find_package(Torch) command above.

The last step is building the application. For this, assume our example directory is laid out like this:

In [25]:
!ls -l

total 45780
-rw-rw-r-- 1 ubuntu ubuntu      260 May 10 18:16 CMakeLists.txt
-rw-rw-r-- 1 ubuntu ubuntu    10519 May 11 02:37 CPP_Export.ipynb
-rw-rw-r-- 1 ubuntu ubuntu 46857749 May 11 02:27 model.pt
-rw-rw-r-- 1 ubuntu ubuntu      468 May 11 02:30 resnet18-app.cpp


We can now run the following commands to build the application from within the `resnet18-app/` folder:

In [45]:
!cmake -DCMAKE_PREFIX_PATH=../../libtorch

-- The C compiler identification is GNU 5.4.0
-- The CXX compiler identification is GNU 5.4.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Looking for pthread_create
-- Looking for pthread_create - not found
-- Looking for pthread_create in pthreads
-- Looking for pthread_create in pthreads - not found
-- Looking for pthread_create in pthread
-- Looking for pthread_create in pthread - found
-- Found Threads: TRUE  
-- Found torch: /home/ubuntu/development/pytorch-lite/libtorch/li

In [46]:
!ls -l

total 45812
-rw-rw-r-- 1 ubuntu ubuntu    13002 May 11 02:43 CMakeCache.txt
drwxrwxr-x 5 ubuntu ubuntu     4096 May 11 02:43 CMakeFiles
-rw-rw-r-- 1 ubuntu ubuntu     1431 May 11 02:43 cmake_install.cmake
-rw-rw-r-- 1 ubuntu ubuntu      260 May 10 18:16 CMakeLists.txt
-rw-rw-r-- 1 ubuntu ubuntu    11696 May 11 02:43 CPP_Export.ipynb
-rw-rw-r-- 1 ubuntu ubuntu     5145 May 11 02:43 Makefile
-rw-rw-r-- 1 ubuntu ubuntu 46857749 May 11 02:27 model.pt
-rw-rw-r-- 1 ubuntu ubuntu      468 May 11 02:30 resnet18-app.cpp


In [47]:
!make

Scanning dependencies of target resnet18-app
[ 50%] Building CXX object CMakeFiles/resnet18-app.dir/resnet18-app.cpp.o
[100%] Linking CXX executable resnet18-app
[100%] Built target resnet18-app


In [50]:
!ls -l resnet18-app

-rwxrwxr-x 1 ubuntu ubuntu 83320 May 11 02:44 resnet18-app


If we supply the path to the serialized `ResNet18` model we created earlier to the resulting `resnet18-app` binary, we should be rewarded with a friendly "ok":

In [52]:
!./resnet18-app

usage: example-app <path-to-exported-script-module>


In [53]:
!./resnet18-app model.pt

ok


## Step 4: Executing the Script Module in C++

Having successfully loaded our serialized `ResNet18` in C++, we are now just a couple lines of code away from executing it! Let’s add those lines to our C++ application’s `main()` function:

In [54]:
!cat resnet18-app.cpp

#include <torch/script.h> // One-stop header.

#include <iostream>
#include <memory>

int main(int argc, const char* argv[]) {
    if (argc != 2) {
        std::cerr << "usage: example-app <path-to-exported-script-module>\n";
        return -1;
    }

    // Deserialize the ScriptModule from a file using torch::jit::load().
    std::shared_ptr<torch::jit::script::Module> module = torch::jit::load(argv[1]);

    assert(module != nullptr);
    std::cout << "ok\n";

    // Create a vector of inputs.
    std::vector<torch::jit::IValue> inputs;
    inputs.push_back(torch::ones({1, 3, 224, 224}));

    // Execute the model and turn its output into a tensor.
    at::Tensor output = module->forward(inputs).toTensor();

    std::cout << output.slice(/*dim=*/1, /*start=*/0, /*end=*/5) << '\n';
}

Let’s try it out by re-compiling our application and running it with the same serialized model:

In [55]:
!make

Scanning dependencies of target resnet18-app
[ 50%] Building CXX object CMakeFiles/resnet18-app.dir/resnet18-app.cpp.o
[100%] Linking CXX executable resnet18-app
[100%] Built target resnet18-app


In [57]:
!./resnet18-app model.pt

ok
 0.0808  0.5042  0.7030 -0.9465  0.4538
[ Variable[CPUFloatType]{1,5} ]


For reference, the output in Python previously was:

```sh
tensor([ 0.0808,  0.5042,  0.7030, -0.9465,  0.4538], grad_fn=<SliceBackward>)
```

Looks like a good match!

## Step 5: Getting Help and Exploring the API

This tutorial has hopefully equipped you with a general understanding of a PyTorch model’s path from Python to C++. With the concepts described in this tutorial, you should be able to go from a vanilla, “eager” PyTorch model, to a compiled `ScriptModule` in Python, to a serialized file on disk and – to close the loop – to an executable `script::Module` in C++.

Of course, there are many concepts we did not cover. To learn more, go to: https://pytorch.org/tutorials/advanced/cpp_export.html#step-5-getting-help-and-exploring-the-api